In [3]:
import pandas as pd
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
#from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras import Sequential
from keras.layers import Dense, Dropout
import keras.metrics
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, TensorBoard
from MLscores import calc_metrics, metrics_dict, cmvals, recall, hybridrecall
import keras.backend as K
import tensorflow as tf
from functools import partial
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import recall_score, make_scorer
from xgboost import XGBClassifier
from check_and_prepare_dataset import load_dataset, prepare_dataset
from manage_model import create_NN_model, create_sklearn_model, allowgrowthgpus
import re
#import eli5
#from eli5.sklearn import PermutationImportance

In [4]:
#params={'ES_mindelta': 0.001, 'ES_monitor': 'loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 10}, 'dropout': 0.1, 'feature_drop': ('dir_max', 'dom_dir', 'month', 'wkd'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 70}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
#params={'ES_mindelta': 0.0001, 'ES_monitor': 'loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': 0.3, 'feature_drop': ('dir_max', 'dom_dir', 'month', 'wkd'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 200.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params = {}
#params['nn_auc']= {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_h1'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 2}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 60.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_h2'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_nh2'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 1700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_nh5'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 10}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 1100.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}

In [5]:
models={}
for paramset in params:
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                         params[paramset]['feature_drop']+ \
                         ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp')) #correlated features
    creatennmodel = partial(create_NN_model, params[paramset], X)
    models[paramset]={}
    models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)
    models[paramset]['params'] = params[paramset]

Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_1', 'bin_weekday_4', 'bin_dom_dir_2', 'bin_dir_max_3', 'bin_dom_dir_8', 'bin_dir_max_6', 'bin_month_5', 'bin_corine_gr21', 'bin_dom_dir_4', 'bin_month_4', 'evi', 'max_dew_temp', 'bin_month_3', 'bin_weekday_3', 'bin_month_9', 'bin_corine_gr33', 'bin_corine_gr32', 'max_temp', 'bin_corine_gr23', 'min_temp', 'bin_dir_max_8', 'bin_corine_gr5', 'dom_vel', 'bin_dom_dir_6', 'bin_dir_max_2', 'bin_dom_dir_3', 'bin_dom_dir_1', 'bin_dom_dir_5', 'min_dew_temp', 'bin_corine_gr4', 'bin_month_6', 'bin_month_8', 'bin_corine_gr31', 'bin_weekday_1', 'bin_corine_gr1', 'bin_weekday_6', 'bin_weekday_2', 'bin_corine_gr22', 'bin_dir_max_5', 'bin_weekday_7', 'bin_weekday_5', 'bin_corine

/tmp/ipykernel_139671/3627430412.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)
/tmp/ipykernel_139671/3627430412.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)


before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_1', 'bin_corine_2', 'bin_corine_14', 'bin_weekday_4', 'bin_corine_11', 'bin_corine_31', 'bin_dir_max_3', 'bin_corine_21', 'bin_corine_4', 'bin_dir_max_6', 'bin_corine_18', 'bin_dom_dir_4', 'bin_corine_40', 'bin_corine_30', 'bin_corine_12', 'bin_corine_10', 'bin_month_9', 'min_temp', 'bin_corine_5', 'bin_dir_max_8', 'bin_corine_26', 'dom_vel', 'bin_corine_44', 'bin_corine_29', 'bin_corine_23', 'bin_dom_dir_1', 'bin_dom_dir_5', 'min_dew_temp', 'bin_month_6', 'bin_weekday_1', 'bin_weekday_6', 'bin_weekday_2', 'bin_dir_max_5', 'bin_corine_27', 'bin_corine_15', 'bin_corine_6', 'bin_dom_dir_7', 'bin_corine_42', 'bin_corine_16', 'bin_corine_17', 'bin_month_10', 'bin_corine_37', 'bin_dom_dir_2', 'bin_corine_43', 'bin_corine_28', 'bin_corine_22', 'bin_

/tmp/ipykernel_139671/3627430412.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)
/tmp/ipykernel_139671/3627430412.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)


In [6]:
paramssk={}
#RF best
paramssk['RF']={'algo': 'RF','bootstrap': True, 'class_weights': {0: 1, 1: 10}, 'criterion': 'entropy', 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_depth': 1000, 'max_features': 2.0, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 350}
#ET best
paramssk['ET']={'algo': 'XT', 'bootstrap': True, 'class_weights': {0: 1, 1: 1}, 'criterion': 'entropy', 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_depth': None, 'max_features': 2.0, 'min_samples_leaf': 50, 'min_samples_split': 150, 'n_estimators': 800}
#XG
paramssk['XGB']={'algo': 'XGB', 'alpha': 60, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'gamma': 0.1, 'lambda': 21.0, 'max_depth': 44.0, 'n_estimators': 640, 'scale_pos_weight': 15, 'subsample': 0.6}
for skalgo in paramssk:
    print(paramssk[skalgo]['algo'])
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                         params[paramset]['feature_drop'] +
                        ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp'))
    models[skalgo]={}
    models[skalgo]['model'] = create_sklearn_model(paramssk[skalgo],X)
    models[skalgo]['params']=paramssk[skalgo]


RF
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_1', 'bin_corine_2', 'bin_corine_14', 'bin_weekday_4', 'bin_corine_11', 'bin_corine_31', 'bin_dir_max_3', 'bin_corine_21', 'bin_corine_4', 'bin_dir_max_6', 'bin_corine_18', 'bin_dom_dir_4', 'bin_corine_40', 'bin_corine_30', 'bin_corine_12', 'bin_corine_10', 'bin_month_9', 'min_temp', 'bin_corine_5', 'bin_dir_max_8', 'bin_corine_26', 'dom_vel', 'bin_corine_44', 'bin_corine_29', 'bin_corine_23', 'bin_dom_dir_1', 'bin_dom_dir_5', 'min_dew_temp', 'bin_month_6', 'bin_weekday_1', 'bin_weekday_6', 'bin_weekday_2', 'bin_dir_max_5', 'bin_corine_27', 'bin_corine_15', 'bin_corine_6', 'bin_dom_dir_7', 'bin_corine_42', 'bin_corine_16', 'bin_corine_17', 'bin_month_10', 'bin

In [7]:
models

{'nn_h1': {'model': <keras.wrappers.scikit_learn.KerasClassifier at 0x7ff519be6fb0>,
  'params': {'ES_mindelta': 0.002,
   'ES_monitor': 'val_loss',
   'ES_patience': 10,
   'batch_size': 512,
   'class_weights': {0: 1, 1: 2},
   'dropout': None,
   'feature_drop': ('month',
    'weekday',
    'dom_dir',
    'dir_max',
    'corine_gr1',
    'corine_gr2',
    'corine_gr3',
    'corine_gr4',
    'corine_gr5',
    'corine_gr6',
    'corine_gr7',
    'corine_gr8',
    'corine_gr9'),
   'max_epochs': 2000,
   'metric': 'accuracy',
   'n_internal_layers': (0, {'layer_1_0_nodes': 60.0}),
   'optimizer': {'adam_params': None, 'name': 'Adam'}}},
 'nn_h2': {'model': <keras.wrappers.scikit_learn.KerasClassifier at 0x7ff3cfa23490>,
  'params': {'ES_mindelta': 0.002,
   'ES_monitor': 'val_loss',
   'ES_patience': 10,
   'batch_size': 512,
   'class_weights': {0: 1, 1: 5},
   'dropout': None,
   'feature_drop': ('month',
    'weekday',
    'dom_dir',
    'dir_max',
    'pop',
    'corine_gr1',
    '

In [8]:
#models1={'RF':models['RF'], 'ET':models['ET']}
allowgrowthgpus()
for modname in models:
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                         models[modname]['params']['feature_drop']+ \
                         ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp'))
    print(modname)
    models[modname]['model'].fit(X,y)
    break

2023-05-23 10:32:40.804112: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-23 10:32:40.908347: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-23 10:32:40.908672: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-23 10:32:40.921479: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other 

1 Physical GPUs, 1 Logical GPUs
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_1', 'bin_weekday_4', 'bin_dom_dir_2', 'bin_dir_max_3', 'bin_dom_dir_8', 'bin_dir_max_6', 'bin_month_5', 'bin_corine_gr21', 'bin_dom_dir_4', 'bin_month_4', 'evi', 'max_dew_temp', 'bin_month_3', 'bin_weekday_3', 'bin_month_9', 'bin_corine_gr33', 'bin_corine_gr32', 'max_temp', 'bin_corine_gr23', 'min_temp', 'bin_dir_max_8', 'bin_corine_gr5', 'dom_vel', 'bin_dom_dir_6', 'bin_dir_max_2', 'bin_dom_dir_3', 'bin_dom_dir_1', 'bin_dom_dir_5', 'min_dew_temp', 'bin_corine_gr4', 'bin_month_6', 'bin_month_8', 'bin_corine_gr31', 'bin_weekday_1', 'bin_corine_gr1', 'bin_weekday_6', 'bin_weekday_2', 'bin_corine_gr22', 'bin_dir_max_5', 'bin_weekday_

2023-05-23 10:32:43.360813: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [9]:
def get_perm_results(models, dataset):
    rec1_scorer = make_scorer(recall_score, pos_label=1)
    rec0_scorer = make_scorer(recall_score, pos_label=0)
    scoring={'recall1':rec1_scorer, 'recall0':rec0_scorer}
    modelsresults=[]
    for modname in models:
        print(modname)
        #models[modname].fit(X,y)
        #ypred=models['nn_auc'].predict(X)
        X, y, g=load_dataset(dataset, \
                              models[modname]['params']['feature_drop']+ \
                             ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp'))
        r=permutation_importance(models[modname]['model'], X, y, n_repeats=3, scoring=scoring)
        modelsresults+=[r]
    return modelsresults

In [20]:
def getcorinedesc(corinefeat, dfcorcats):
    r=re.search(r"\d+$",corinefeat)
    corid=r.group(0)
    corstr=dfcorcats.loc[dfcorcats['Unnamed: 3']==corid,'CORINE LAND COVER LEGEND'].values[0]
    return corstr

def colcond1(X,i):
    return not any(c in X.columns[i] for c in ['dir_max','dom_dir','wkd','month','corine'])


def colcond2(X,i):
    return 'corine' in X.columns[i]
    
#for r in modelsresults
def resdict(condfunc, modelsresult):
    results_lists={}
    for s in modelsresult:
        results_lists['%s %s'%(modname,s)]=[]
        cnt=0
        for i in modelsresult[s].importances_mean.argsort()[::-1]:
            #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
            cnt+=1
            if condfunc(X,i):
                results_dict={}
                results_dict['rank %s %s'%(modname,s)]=cnt
                results_dict['feat %s %s'%(modname,s)]=X.columns[i]
                results_dict['perm. imp. %s %s'%(modname,s)]=modelsresult[s].importances_mean[i]
                results_lists['%s %s'%(modname,s)]+=[results_dict]
    return results_lists

def get_perm_pandas(cond, modelsresult):
    results_lists=resdict(cond, modelsresult)
    results_pd=[]
    for mod in results_lists:
        results_pd+=[pd.DataFrame(results_lists[mod])]
    all_res_pd=pd.concat(results_pd, axis=1)
    return all_res_pd

In [11]:
m1=next(iter(models.items()))
permres=get_perm_results({m1[0]:m1[1]}, '/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv')

nn_h1
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_1', 'bin_weekday_4', 'bin_dom_dir_2', 'bin_dir_max_3', 'bin_dom_dir_8', 'bin_dir_max_6', 'bin_month_5', 'bin_corine_gr21', 'bin_dom_dir_4', 'bin_month_4', 'evi', 'max_dew_temp', 'bin_month_3', 'bin_weekday_3', 'bin_month_9', 'bin_corine_gr33', 'bin_corine_gr32', 'max_temp', 'bin_corine_gr23', 'min_temp', 'bin_dir_max_8', 'bin_corine_gr5', 'dom_vel', 'bin_dom_dir_6', 'bin_dir_max_2', 'bin_dom_dir_3', 'bin_dom_dir_1', 'bin_dom_dir_5', 'min_dew_temp', 'bin_corine_gr4', 'bin_month_6', 'bin_month_8', 'bin_corine_gr31', 'bin_weekday_1', 'bin_corine_gr1', 'bin_weekday_6', 'bin_weekday_2', 'bin_corine_gr22', 'bin_dir_max_5', 'bin_weekday_7', 'bin_weekday_5', 'bin_

NameError: name 'scoring' is not defined

In [13]:
permres

[{'recall1': {'importances_mean': array([ 1.34583033e-03,  1.03340543e-03, -4.80653689e-05,  7.20980534e-05,
           6.00817111e-03, -1.08147080e-03,  4.08555636e-04,  8.43547224e-03,
           8.41143956e-04,  3.78755107e-02,  1.44196107e-04,  2.07642394e-02,
           1.44196107e-03,  2.12208604e-02,  1.97068012e-03,  5.14780101e-02,
           2.24945926e-02,  2.76135544e-02,  0.00000000e+00,  1.70632060e-03,
           7.68084595e-02,  6.38788753e-02,  1.82648402e-03,  7.20980534e-05,
           7.45013218e-03,  8.93775535e-02,  2.33357366e-02,  9.68997837e-02,
           6.48882480e-04,  1.05383321e-01,  2.35904831e-01,  3.38860851e-03,
           9.61307378e-04,  1.24969959e-03,  1.92261476e-04,  2.74212930e-02,
           1.10742610e-01,  3.58086998e-03,  4.09516943e-02,  4.62388849e-02,
           7.45013218e-04,  4.85460226e-03,  3.33093006e-02,  2.62436914e-02,
           2.40326845e-05,  1.20163422e-04,  2.04277818e-03,  9.85340062e-04,
           2.13890892e-03,  1.490

In [15]:
all_res_pd=get_perm_pandas(colcond1, permres[0])

In [15]:
#for i in r.importances_mean.argsort()[::-1]:
    #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#    print(f"{X.columns[i]:<17}"
#        f"{r.importances_mean[i]:.3f}"
#        f" +/- {r.importances_std[i]:.3f}")

In [16]:
all_res_pd

,rank nn_h1 recall1,feat nn_h1 recall1,perm. imp. nn_h1 recall1,rank nn_h1 recall0,feat nn_h1 recall0,perm. imp. nn_h1 recall0
0,1,res_max,0.235905,2,ypos,0.044390
1,3,f81,0.110743,4,mean_dew_temp,0.034203
2,4,mean_temp,0.105383,5,res_max,0.029529
3,7,dem,0.076808,7,f81,0.020769
4,9,xpos,0.051478,8,xpos,0.017497
5,12,rain_7_days,0.040952,12,dem,0.014968
6,17,ndvi,0.026244,20,lst_night,0.004842
7,18,mean_dew_temp,0.023336,21,ndvi,0.003416
8,20,road_dens,0.021221,23,frequency,0.002541
9,22,ypos,0.014900,26,road_dens,0.002121


In [21]:
results_lists=resdict(colcond2, permres[0])
results_pd=[]
for mod in results_lists:
    results_pd+=[pd.DataFrame(results_lists[mod])]
all_res_pd2=pd.concat(results_pd, axis=1)
dfcorcats=pd.read_csv('/mnt/nvme2tb/ffp/code/mlfires/ML_fires_al/corine_categories.csv')
all_res_pd2['feat nn_h1 recall0']=all_res_pd2['feat nn_h1 recall0'].apply(getcorinedesc, args=(dfcorcats,))
all_res_pd2['feat nn_h1 recall1']=all_res_pd2['feat nn_h1 recall1'].apply(getcorinedesc, args=(dfcorcats,))

In [90]:
all_res_pd2

,rank nn_h1 recall1,feat nn_h1 recall1,perm. imp. nn_h1 recall1,rank nn_h1 recall0,feat nn_h1 recall0,perm. imp. nn_h1 recall0
0,2,3.2.3 Sclerophyllous vegetation,0.184331,1,3.2.3 Sclerophyllous vegetation,0.044270
1,4,3.2.1 Natural grasslands,0.106777,3,3.2.1 Natural grasslands,0.030308
2,6,3.2.4 Transitional woodland-shrub,0.094833,6,3.2.4 Transitional woodland-shrub,0.023094
3,8,2.4.3 Land principally occupied by agriculture...,0.066643,8,3.3.4 Burnt areas,0.016574
4,10,3.3.4 Burnt areas,0.048354,9,2.4.3 Land principally occupied by agriculture...,0.012883
5,11,2.2.3 Olive groves,0.044773,10,3.1.3 Mixed forest,0.012512
6,14,2.4.2 Complex cultivation patterns,0.036866,11,3.1.2 Coniferous forest,0.012176
7,15,2.1.1 Non-irrigated arable land,0.035184,12,3.1.1 Broad-leaved forest,0.011014
8,16,3.3.3 Sparsely vegetated areas,0.031435,13,3.3.3 Sparsely vegetated areas,0.010834
9,17,3.1.2 Coniferous forest,0.026196,14,2.1.1 Non-irrigated arable land,0.009923


In [22]:
all_res_pd.to_csv('/mnt/nvme2tb/ffp/results/newdefCV/permimpcorine.csv')
all_res_pd2.to_csv('/mnt/nvme2tb/ffp/results/newdefCV/permimpmainfeat.csv')

In [119]:
m1=next(iter(models.items()))
modres=get_perm_results({m1[0]:m1[1]}, '/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_attica_norm.csv')

nn_h1
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_attica_norm.csv
before nan drop: 2055
after nan drop: 2055
after dup. drop: 2054
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['evi', 'bin_corine_gr31', 'bin_month_7', 'bin_weekday_4', 'bin_dom_dir_1', 'min_temp', 'bin_dom_dir_5', 'bin_dom_dir_4', 'bin_dir_max_6', 'bin_month_4', 'max_dew_temp', 'bin_corine_gr23', 'bin_dir_max_5', 'bin_corine_gr4', 'bin_month_10', 'bin_dir_max_3', 'min_dew_temp', 'bin_corine_gr22', 'bin_dom_dir_6', 'bin_corine_gr5', 'bin_weekday_7', 'bin_weekday_6', 'bin_dom_dir_8', 'dom_vel', 'bin_dir_max_4', 'bin_dom_dir_2', 'bin_corine_gr32', 'bin_corine_gr33', 'bin_weekday_1', 'bin_month_8', 'bin_dir_max_2', 'max_temp', 'bin_corine_gr21', 'bin_weekday_2', 'bin_weekday_5', 'bin_dom_dir_3', 'bin_month_3', 'bin_corine_gr1', 'bin_month_6', 'bin_dir_max_1', 'bin_month_9', 'b

65/65 [==============================] - 0s 342us/step


In [127]:
dfresgen=get_perm_pandas(colcond1, modres[0])
dfresgen

,rank nn_h1 recall1,feat nn_h1 recall1,perm. imp. nn_h1 recall1,rank nn_h1 recall0,feat nn_h1 recall0,perm. imp. nn_h1 recall0
0,4,res_max,0.075623,1,res_max,0.136695
1,6,pop,0.066941,2,mean_temp,0.076190
2,7,f81,0.065113,4,dem,0.038095
3,15,mean_dew_temp,0.023304,6,pop,0.032493
4,16,ypos,0.022618,8,road_dens,0.024090
5,17,mean_temp,0.022390,12,ndvi,0.011765
6,23,xpos,0.008682,13,aspect,0.011765
7,24,dem,0.005940,17,mean_dew_temp,0.008403
8,27,aspect,0.002970,20,lst_day,0.006723
9,28,frequency,0.002970,21,slope,0.004482


In [128]:
dfrescor=get_perm_pandas(colcond2, modres[0])
dfrescor['feat nn_h1 recall0']=dfrescor['feat nn_h1 recall0'].apply(getcorinedesc)
dfrescor['feat nn_h1 recall1']=dfrescor['feat nn_h1 recall1'].apply(getcorinedesc)
dfrescor

,rank nn_h1 recall1,feat nn_h1 recall1,perm. imp. nn_h1 recall1,rank nn_h1 recall0,feat nn_h1 recall0,perm. imp. nn_h1 recall0
0,1,3.2.4 Transitional woodland-shrub,0.129769,3,3.3.4 Burnt areas,0.040896
1,2,3.2.3 Sclerophyllous vegetation,0.123144,5,3.2.4 Transitional woodland-shrub,0.035294
2,3,3.1.2 Coniferous forest,0.078136,7,2.4.3 Land principally occupied by agriculture...,0.027451
3,5,2.4.3 Land principally occupied by agriculture...,0.067398,9,1.1.2 Discontinuous urban fabric,0.023529
4,8,3.3.3 Sparsely vegetated areas,0.050948,10,2.4.2 Complex cultivation patterns,0.015686
5,9,3.3.4 Burnt areas,0.050491,11,3.2.1 Natural grasslands,0.012885
6,10,2.4.2 Complex cultivation patterns,0.050263,14,3.1.3 Mixed forest,0.010644
7,11,2.1.1 Non-irrigated arable land,0.043866,15,3.1.2 Coniferous forest,0.010644
8,12,1.1.2 Discontinuous urban fabric,0.043180,16,3.3.3 Sparsely vegetated areas,0.008964
9,13,3.1.3 Mixed forest,0.027873,18,3.3.2 Bare rocks,0.006723
